In [6]:
import gspread
import json
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy.signal import argrelextrema
import os

import sys
sys.path.append('../../')
from cad.calc.geo import Geo
from cad.calc.conv import freq_to_note_and_cent, note_name
import pandas as pd
import seaborn as sns




In [7]:
# get peaks in fft
def get_n_peaks(freq, spectrum, n=10):
    # get all maxima
    peaks = []
    rising = True
    
    #spectrum = [0,1,2,1,0,1,0]
    #freq = np.arange(len(spectrum))
    
    for i in range(1,len(spectrum)):
        if spectrum[i]<spectrum[i-1] and rising:
            rising = False
            peaks.append(i-1)
        if spectrum[i]>spectrum[i-1]:
            rising = True
            
    # get n highest maxima
    peaks = sorted(peaks, key=lambda x : spectrum[x], reverse=True)
    df = []
    for i in range(n):
        f = freq[peaks[i]]
        note, cent = freq_to_note_and_cent(f)
        imp = spectrum[peaks[i]]
        
        skip = False
        for j in range(len(df)):
            if np.abs(np.log2(df[j][2])-np.log2(f))<0.15:
                if imp>df[j][3]:
                    del df[j]
                    break
                else:
                    skip = True
                    break
        
        if not skip:
            df.append([note_name(note), cent, f, imp])
    
    df = pd.DataFrame(df, columns=["note", "cent-diff", "freq", "impedance"])
    
    
    
    return df



def do_fft(infile, size=-1, maxfreq=1000):
    sampFreq, sound = wavfile.read(infile)
    
    if len(sound.shape)==2:
        signal = sound[:,0]
    else:
        signal = sound

    if size == -1:
        size=len(signal)
    
    fft_spectrum = np.fft.rfft(signal, n=size)
    freq = np.fft.rfftfreq(size, d=1./sampFreq)
    fft_spectrum_abs = np.abs(fft_spectrum)
 
    i=0
    while i<len(freq) and freq[i]<=maxfreq:
        i+=1
    freq = freq[0:i]
    fft_spectrum_abs = fft_spectrum_abs[0:i]
    #fft_spectrum_abs = np.log2(fft_spectrum_abs)

    return freq, fft_spectrum_abs




In [ ]:

credentials = json.load(open("service_account.json", "r"))

gc = gspread.service_account("service_account.json")

# Open a sheet from a spreadsheet in one go
wks = gc.open_by_key("16_tkHuyBDrFTOWlu4-UH1QM4oBGv72GPem8N4IvthxI").sheet1

# Update a range of cells using the top left corner address
wks.update('A1', [[1, 2], [3, 4]])

# Or update a single cell
wks.update('B42', "it's down there somewhere, let me take another look.")

# Format the header
wks.format('A1:B1', {'textFormat': {'bold': True}})



In [61]:
# generate all frequency spectra images
import shutil
outfolder = "/tmp/didgelab-images/"
if os.path.exists(outfolder):
    shutil.rmtree(outfolder)
os.mkdir(outfolder)

for i in range(len(infiles)):
    infile = infiles[i]
    freq, spectrum = do_fft(os.path.join(path, infile))
    spectrum = np.log2(spectrum)
    plt.plot(freq, spectrum, label="sound")

    plt.xlim((0,1000))

    plt.xlabel("Frequency, Hz")
    plt.ylabel("Amplitude")
    # plt.show()
    
    
    outfile = os.path.join(outfolder, infile + ".png")
    plt.savefig(outfile)
    print(outfile)
    plt.clf()


/tmp/didgelab-images/1.wav.png
/tmp/didgelab-images/2.wav.png
/tmp/didgelab-images/3.wav.png
/tmp/didgelab-images/4.wav.png
/tmp/didgelab-images/5.wav.png
/tmp/didgelab-images/6.wav.png
/tmp/didgelab-images/7.wav.png
/tmp/didgelab-images/9.wav.png
/tmp/didgelab-images/dennis1st.wav.png
/tmp/didgelab-images/dennis2nd.wav.png
/tmp/didgelab-images/dennisCh.wav.png
/tmp/didgelab-images/dennisCl.wav.png
/tmp/didgelab-images/dennisCm.wav.png
/tmp/didgelab-images/pvc_CSharp_38_24.wav.png
/tmp/didgelab-images/trevor1st.wav.png
/tmp/didgelab-images/trevor2nd.wav.png
/tmp/didgelab-images/trevorCh.wav.png
/tmp/didgelab-images/trevorCl.wav.png
/tmp/didgelab-images/trevorCm.wav.png


<Figure size 640x480 with 0 Axes>

In [123]:
# do google authentication

from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
   
# For using listdir()
import os
   
  
# Below code does the authentication
# part of the code
gauth = GoogleAuth()
  
# Creates local webserver and auto
# handles authentication.
gauth.LocalWebserverAuth()       
drive = GoogleDrive(gauth)


Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=616465563771-cvta8n3p1kl31lee0eb7ik5lakq8uc2o.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code



KeyboardInterrupt: 

In [140]:
# didgelab database to table

gc = gspread.service_account("service_account.json")

# Open a sheet from a spreadsheet in one go
wks = gc.open_by_key("16_tkHuyBDrFTOWlu4-UH1QM4oBGv72GPem8N4IvthxI").sheet1

# get a list of all mp3 files
file_list = drive.ListFile({'q': "'1cYniLENeVXueqzO-gQCkawXk3UcSiwtU' in parents and trashed=false"}).GetList()
mp3_files_to_ids = {x.attr["metadata"]["title"][0:-4]: x.attr["metadata"]["id"] for x in file_list}
mp3_files_to_ids

{'trevorCh': '1ZjSZLjpKisMI4_UQg1g7irD0hIYMseWk',
 'trevorCm': '1vnFuq6vjpJsGvnCMXonhVdIcIa4jV0hY',
 'trevorCl': '1aWDJH1_e0YrgRqfnq8uZUtdHxRvCJzrf',
 'trevor1st': '1e9Vuxwx4Xli5tf2A08PiORdm083lKai1',
 'trevor2nd': '1YzsVlmFkJbr5Z-Ws7xc0xXm9D2--KSx7',
 'pvc_CSharp_38_24': '1WtK_ML7Ao5cUkJ_tUnGlkxgFsTdJ1bdz',
 'dennisCh': '1os8aLJ-w8gKwIGoS7QDnkg1jGKfN9UmH',
 'dennisCm': '1Y4iIXBDsHHfFpdU9Oi8JiDGuzOPXHrmP',
 'dennisCl': '1yHmKZyuz3QKJnOIoeDqIcixXYbH7Liki',
 'dennis1st': '1-r7WpD6iranEHm53BZfpiPV9LQDd8NNG',
 'dennis2nd': '1N8ZM8DYFmk-dXWS6Yd0GYpOeO1D1AdPk',
 '9': '15RHLKde42UFPdwHqko2-pmFhvjZWUhZs',
 '7': '1Q4ZAwKF3fd2o2cYGRdkbTut4XugZKCP-',
 '6': '154KRUB0O4pVHLNqj5nXUl0AUWO-4F0ww',
 '5': '12ZFhkW5058SFkDRc4chJyjb88bstjDhC',
 '4': '1C-_v_SmrFlGvr7_Gk8PWz_A6tFbN5244',
 '3': '10cRjtpcJWiWO0Qus9xr9lGhRLJOPm8Fg',
 '2': '1eIMXrG1oTbW7zqHamzC1841UYUXLjmIE',
 '1': '1OP7xNamrNNI1GoJQX1OwJcp8fBTYorlX'}

In [134]:
file_list = drive.ListFile({'q': "'1yVCjCfgWkp_FX5sb-KIMu9-kA5R8WDbL' in parents and trashed=false"}).GetList()
image_files = {x.attr["metadata"]["title"]: x.attr["metadata"]["id"] for x in file_list}
image_files

{'trevorCm.wav.png': '17wWqK21mTPRPa00wjDx1GELqjMvtP-i3',
 'trevorCl.wav.png': '1f18o0P4Mm0MiKJAVAP7eUemNkysaBkjV',
 'trevorCh.wav.png': '1zcZa-GjOyv0rjRxaFLKAJmOebwwoAfpK',
 'trevor2nd.wav.png': '1W3tVWz0IJ1fIfLAGf9Y5ejD-7aIeZ8Hq',
 'trevor1st.wav.png': '1FtncA35sp1rEF2nrUBoiauV3Lw0wah_R',
 'pvc_CSharp_38_24.wav.png': '1Ft3R69w_Ezy17GBdcoyV3pprU-M3sJGt',
 'dennisCm.wav.png': '1xGlZogW9E8Nn7CGUm5DKXuobEDXNvBoW',
 'dennisCl.wav.png': '1ZCp-HCIdj1kIDjC6-nnfS4VUze-gWuk8',
 'dennisCh.wav.png': '1dZeJ_VUSL8fyS5gJ1gYhf0Kz6wW3dHUi',
 'dennis2nd.wav.png': '193xxhfLu9aNo5HFRdnvlsGPth3VeO_OA',
 'dennis1st.wav.png': '1JNJtQJui3B4YlBKF5Bn8aVkRcYqpx7h8',
 '9.wav.png': '1-Os-gdPG1Q9RcGDxrPUKjfSC9YvBNrfw',
 '7.wav.png': '1q0vkK1FuBfREcedr8doRDFW_xHrJlQti',
 '6.wav.png': '136EK5Qe2awG-xs4ZnwtUS8yLst5Jx6JG',
 '5.wav.png': '1RiplCcHlF3Hc1ClpBiGuCJ05DOkcm-5-',
 '4.wav.png': '1MrJOqqwseKyGmg9WvjIYDw46zLovQ_fB',
 '3.wav.png': '13ZVSu2xV6srbTTwDf2HhdR9Mjei7JdHr',
 '2.wav.png': '1Y9XQyanGgrLy48TDmB-eX38dX5Tc

{'trevorCh.mp3': '1ZjSZLjpKisMI4_UQg1g7irD0hIYMseWk',
 'trevorCm.mp3': '1vnFuq6vjpJsGvnCMXonhVdIcIa4jV0hY',
 'trevorCl.mp3': '1aWDJH1_e0YrgRqfnq8uZUtdHxRvCJzrf',
 'trevor1st.mp3': '1e9Vuxwx4Xli5tf2A08PiORdm083lKai1',
 'trevor2nd.mp3': '1YzsVlmFkJbr5Z-Ws7xc0xXm9D2--KSx7',
 'pvc_CSharp_38_24.mp3': '1WtK_ML7Ao5cUkJ_tUnGlkxgFsTdJ1bdz',
 'dennisCh.mp3': '1os8aLJ-w8gKwIGoS7QDnkg1jGKfN9UmH',
 'dennisCm.mp3': '1Y4iIXBDsHHfFpdU9Oi8JiDGuzOPXHrmP',
 'dennisCl.mp3': '1yHmKZyuz3QKJnOIoeDqIcixXYbH7Liki',
 'dennis1st.mp3': '1-r7WpD6iranEHm53BZfpiPV9LQDd8NNG',
 'dennis2nd.mp3': '1N8ZM8DYFmk-dXWS6Yd0GYpOeO1D1AdPk',
 '9.mp3': '15RHLKde42UFPdwHqko2-pmFhvjZWUhZs',
 '7.mp3': '1Q4ZAwKF3fd2o2cYGRdkbTut4XugZKCP-',
 '6.mp3': '154KRUB0O4pVHLNqj5nXUl0AUWO-4F0ww',
 '5.mp3': '12ZFhkW5058SFkDRc4chJyjb88bstjDhC',
 '4.mp3': '1C-_v_SmrFlGvr7_Gk8PWz_A6tFbN5244',
 '3.mp3': '10cRjtpcJWiWO0Qus9xr9lGhRLJOPm8Fg',
 '2.mp3': '1eIMXrG1oTbW7zqHamzC1841UYUXLjmIE',
 '1.mp3': '1OP7xNamrNNI1GoJQX1OwJcp8fBTYorlX'}

In [146]:
path = "didgeridoo-samples/wav_short"
infiles = sorted(os.listdir(path))
infiles = list(filter(lambda x : x != ".DS_Store" and os.path.isfile(os.path.join(path, x)), infiles))

headings = ['ID', 'File', 'Fundamental', 'Peaks', 'Spektrum']
data = [headings]

for i in range(len(infiles)):

    infile = infiles[i]
    freq, spectrum = do_fft(os.path.join(path, infile))
    peaks = get_n_peaks(freq, spectrum)
    peaks = peaks[["note", "cent-diff", "freq"]]
    print(peaks)

    
    for key in ("cent-diff", "freq"):
        peaks[key] = peaks[key].apply(lambda x : f"{x:.2f}")
                                      
    # fft_spectrum_abs = np.log2(fft_spectrum_abs)
    
    href = f"=HYPERLINK(\"https://drive.google.com/file/d/{mp3_files_to_ids[infile[0:-4]]}\"; \"{infile}\")"
                 
    
    #image_link = "https://drive.google.com/file/d/" + image_files[infile + ".png"] + "/view?usp=drive_link"
    #image_file = f"=image(\"{image_link}\")"                              
                                      
    image_file = "=image(\"https://jannehring.de/didgelab/images/" + infile + ".png\")"
    row = [i, href, peaks.iloc[0]["note"], peaks.to_string(), image_file]
    data.append(row)

wks.update('A1', data)

files = [[x[1]] for x in data[1:]]
wks.update('B2', files, value_input_option='USER_ENTERED')
images = [[x[4]] for x in data[1:]]
wks.update('E2', images, value_input_option='USER_ENTERED')


#wks.update('A1', data)
wks.format('A1:G1', {'textFormat': {'bold': True}})


  note  cent-diff        freq
0  F#1   3.576889   92.307692
1  F#4  19.103756  731.868132
2  C#3  15.416655  274.725275
3  F#3  13.912646  367.032967
4  G#4  13.461654  824.175824
5  A#4  25.526838  459.340659
6  F#2   3.576889  184.615385
7  D#4 -47.492615  639.560440
  note  cent-diff        freq
0   E1  26.807615   81.140673
1   E2  26.807615  162.281346
2   E3  17.424210  326.326621
3  G#3  40.493901  405.703366
4  F#4  14.554301  733.793914
5   B5  21.719158  975.452006
6   E4  22.109555  650.889314
7  C#1  12.599300   68.793179
  note  cent-diff        freq
0   D1 -15.930307   74.094864
1   A4 -28.345968  447.263542
2   A3 -17.885308  222.284592
3  F#3  -8.527999  371.821499
4   C4   6.273642  521.358406
5   E4 -26.821100  669.548134
6   D4 -15.930307  592.758911
7   D3 -23.781722  297.726635
  note  cent-diff        freq
0  F#1  10.976305   91.914005
1  F#2   4.430989  184.524328
2  F#3   7.700553  368.352337
3  A#4  22.041494  460.266342
4  A#5  20.732432  921.229002
5  C#4   6

/Users/jane03/miniconda3/envs/didge/lib/python3.10/site-packages/gspread/worksheet.py:1046: UserWarning: [Deprecated][in version 6.0.0]: method signature will change to: 'Worksheet.update(value = [[]], range_name=)' arguments 'range_name' and 'values' will swap, values will be mandatory of type: 'list(list(...))'
  warnings.warn(


{'spreadsheetId': '16_tkHuyBDrFTOWlu4-UH1QM4oBGv72GPem8N4IvthxI',
 'replies': [{}]}

In [79]:
for key in ("cent-diff", "freq", "impedance"):
    peaks[key] = peaks[key].apply(lambda x : f"{x:.2f}")

In [35]:
file_list = drive.ListFile({'q': "'1yVCjCfgWkp_FX5sb-KIMu9-kA5R8WDbL' in parents and trashed=false"}).GetList()
title_to_id = {x.attr["metadata"]["title"]: x.attr["metadata"]["id"] for x in file_list}
title_to_id

{'trevorCm.wav.png': '17wWqK21mTPRPa00wjDx1GELqjMvtP-i3',
 'trevorCl.wav.png': '1f18o0P4Mm0MiKJAVAP7eUemNkysaBkjV',
 'trevorCh.wav.png': '1zcZa-GjOyv0rjRxaFLKAJmOebwwoAfpK',
 'trevor2nd.wav.png': '1W3tVWz0IJ1fIfLAGf9Y5ejD-7aIeZ8Hq',
 'trevor1st.wav.png': '1FtncA35sp1rEF2nrUBoiauV3Lw0wah_R',
 'pvc_CSharp_38_24.wav.png': '1Ft3R69w_Ezy17GBdcoyV3pprU-M3sJGt',
 'dennisCm.wav.png': '1xGlZogW9E8Nn7CGUm5DKXuobEDXNvBoW',
 'dennisCl.wav.png': '1ZCp-HCIdj1kIDjC6-nnfS4VUze-gWuk8',
 'dennisCh.wav.png': '1dZeJ_VUSL8fyS5gJ1gYhf0Kz6wW3dHUi',
 'dennis2nd.wav.png': '193xxhfLu9aNo5HFRdnvlsGPth3VeO_OA',
 'dennis1st.wav.png': '1JNJtQJui3B4YlBKF5Bn8aVkRcYqpx7h8',
 '9.wav.png': '1-Os-gdPG1Q9RcGDxrPUKjfSC9YvBNrfw',
 '7.wav.png': '1q0vkK1FuBfREcedr8doRDFW_xHrJlQti',
 '6.wav.png': '136EK5Qe2awG-xs4ZnwtUS8yLst5Jx6JG',
 '5.wav.png': '1RiplCcHlF3Hc1ClpBiGuCJ05DOkcm-5-',
 '4.wav.png': '1MrJOqqwseKyGmg9WvjIYDw46zLovQ_fB',
 '3.wav.png': '13ZVSu2xV6srbTTwDf2HhdR9Mjei7JdHr',
 '2.wav.png': '1Y9XQyanGgrLy48TDmB-eX38dX5Tc